In [ ]:
# Ingredients to be used in 10 recipes

# ["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]
# ["bay scallops", "butter", "onions", "parsley", "lemons", "bread crumbs"]
# ["rice", "chicken", "cream of mushroom soup", "cream of chicken soup", "onion soup", "water"]
# ["coriander", "allspice", "cumin", "ground ginger", "turmeric", "cinnamon", "cayenne", "pepper", "salt", "lemon juice", "olive oil", "garlic", "chicken", "pita", "red onion", "parsley", "shredded red cabbage", "tahini"]
# ["sour cream", "heavy cream", "baking powder", "kosher salt", "sugar", "ground black pepper", "flour", "butter", "salt", "cutter"]
# ["hamburger", "egg", "bread crumbs", "onion", "milk", "salt", "pepper", "bacon", "catsup"]
# ["stew meat", "cabbage", "onion", "green bell peppers", "oil", "tomatoes", "garlic", "soy sauce"]
# ["chicken broth", "water", "chicken", "onion", "vegetables", "garlic", "salt", "fresh cilantro", "potatoes", "vegetable oil"]
# ["pork sparerib", "black bean sauce", "Chinese rice wine", "cornstarch", "ginger", "garlic", "ground black pepper", "cooking oil", "sesame oil", "sugar", "red chili pepper"]
# ["shell pasta", "red pepper", "green onion", "seafood", "peas", "olive oil", "sesame oil", "rice vinegar", "lemon", "mayonnaise"]

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader